In [ ]:
!pip install -U pycaret

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from pycaret.classification import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('/kaggle/input/driving-behavior/train_motion_data.csv')
df_test = pd.read_csv('/kaggle/input/driving-behavior/test_motion_data.csv')

In [ ]:
df_train['Class'] = df_train['Class'].replace(['NORMAL', 'SLOW', 'AGGRESSIVE'], [1, 0, 1])
df_test['Class'] = df_test['Class'].replace(['NORMAL', 'SLOW', 'AGGRESSIVE'], [1, 0, 1])

In [ ]:
df_train = shuffle(df_train, random_state=42)
df_test = shuffle(df_test, random_state=42)

In [ ]:
import plotly.express as px

# bar plot
fig = px.histogram(df_train, x='Class', nbins=3, color='Class')
fig.show()

# box plots
fig = px.box(df_train, x='Class', y='AccX', color='Class')
fig.show()

fig = px.box(df_train, x='Class', y='AccY', color='Class')
fig.show()

fig = px.box(df_train, x='Class', y='AccZ', color='Class')
fig.show()

fig = px.box(df_train, x='Class', y='GyroX', color='Class')
fig.show()

fig = px.box(df_train, x='Class', y='GyroY', color='Class')
fig.show()

fig = px.box(df_train, x='Class', y='GyroZ', color='Class')
fig.show()

In [ ]:
import plotly.express as px

fig1 = px.bar(df_train.groupby('Class').size().reset_index(name='Count'), x='Class', y='Count', labels={'Class': 'Class', 'Count': 'Count'})
fig1.update_layout(title='Class Distribution')
fig1.show()

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(df_train, dimensions=['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ'], color='Class')
fig.update_layout(title='Pairplot of Features')
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(z=df_train.corr(), x=df_train.columns, y=df_train.columns, colorscale='Blues', zmin=-1, zmax=1))
fig.update_layout(title='Correlation Heatmap')
fig.show()

In [ ]:
df_train = df_train.drop(['Timestamp'], axis=1)
df_test = df_test.drop(['Timestamp'], axis=1)

In [ ]:
train_data, val_data = train_test_split(df_train, test_size=0.2, random_state=42)

In [ ]:
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_val = val_data.iloc[:, :-1].values
y_val = val_data.iloc[:, -1].values
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

The model has been trained on a dataset with a training set accuracy of 62.13% and a validation set accuracy of 65.98%. The loss function used during training is the binary cross-entropy loss, and the optimizer used is the default one in Keras, which is Adam.

The training accuracy starts at 52.69% and gradually improves to 62.13% after 50 epochs. The validation accuracy starts at 63.79% and reaches a peak of 66.67% at epoch 3, then fluctuates between 65.98% and 66.67% for the remaining epochs.

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=64)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

The test loss and accuracy are reported as the final performance metrics of the model. The test loss is 0.6577 and the test accuracy is 0.5879, which indicates that the model did not perform well on the test data.

In [ ]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

# Add training loss trace
fig.add_trace(
    go.Scatter(x=[0], y=[history.history['loss'][0]], name='Training Loss', mode='lines', line=dict(color='blue'))
)

# Add validation loss trace
fig.add_trace(
    go.Scatter(x=[0], y=[history.history['val_loss'][0]], name='Validation Loss', mode='lines', line=dict(color='orange'))
)

# Set up layout
fig.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    template='plotly_white',
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        buttons=[dict(
            label='Play',
            method='animate',
            args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True)]
        ), dict(
            label='Pause',
            method='animate',
            args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate', transition=dict(duration=0))]
        )]
    )]
)

# Define animation frames
frames = [go.Frame(
    data=[
        go.Scatter(x=list(range(epoch+1)), y=history.history['loss'][:epoch+1], mode='lines', line=dict(color='blue')),
        go.Scatter(x=list(range(epoch+1)), y=history.history['val_loss'][:epoch+1], mode='lines', line=dict(color='orange'))
    ]
) for epoch in range(len(history.history['loss']))]

# Add frames to animation
fig.frames = frames

# Show figure
fig.show()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Training Accuracy", "Validation Accuracy"))

# Add traces for training and validation accuracy
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['accuracy'], mode='lines', name='Training Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_accuracy'], mode='lines', name='Validation Accuracy'), row=1, col=2)

# Set up animation frames
frames = [go.Frame(data=[go.Scatter(x=history.epoch[:i+1], y=history.history['accuracy'][:i+1], mode='lines', name='Training Accuracy'),
                         go.Scatter(x=history.epoch[:i+1], y=history.history['val_accuracy'][:i+1], mode='lines', name='Validation Accuracy')])
         for i in range(len(history.epoch))]

# Add animation to figure
fig.update(frames=frames)
fig.update_layout(updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None,
                                                        {'frame': {'duration': 1000, 'redraw': True},
                                                         'fromcurrent': True, 'transition': {'duration': 0}}]),
                                             dict(label='Pause',
                                                  method='animate',
                                                  args=[[None], {'frame': {'duration': 0, 'redraw': False},
                                                                 'mode': 'immediate',
                                                                 'transition': {'duration': 0}}])])])

# Set animation layout properties
fig.update_layout(title='Training and Validation Accuracy',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy',
                  showlegend=False,
                  width=1000,
                  height=500)

fig.show()

In [ ]:
fig = px.scatter(x=[0], y=[0], text=['Thank You!'], size=[1000])
fig.update_traces(textfont_size=100, textfont_color='black')
fig.update_layout(
    width=800, height=800,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
)
fig.show()